In [ ]:
import os,sys
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import tensorflow as tf
import numpy as np
from keras import backend as K
import tables
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
%load_ext autoreload
%autoreload 2

In [ ]:
# Directories for models
custom_weights_dir_q = os.path.expanduser("weights-quantized-224x224-fixval-best-final/")
custom_weights_dir = os.path.expanduser("weights-floatingpoint-224x224-fixval-best/")
saved_model_dir = os.path.expanduser("models")

In [ ]:
# imported from utils 
# utils.py: https://github.com/nhanvtran/MachineLearningNotebooks/blob/nvt/bwcustomweights-validate/project-brainwave/utils.py
def count_events(train_files):
    import tables
    n_events = 0
    for train_file in train_files:
        f = tables.open_file(train_file, 'r')
        n_events += f.root.label.shape[0]
        f.close()
    return n_events

def preprocess_images(size=64):
    import tensorflow as tf
    # Create a placeholder for our incoming images
    in_height = size
    in_width = size
    in_images = tf.placeholder(tf.float32)
    in_images.set_shape([None, in_height, in_width, 3])
    
    # Resize those images to fit our featurizer
    if size==64:
        out_width = 224
        out_height = 224
        image_tensors = tf.image.resize_images(in_images, [out_height,out_width])
        image_tensors = tf.to_float(image_tensors)
    elif size==224:
        image_tensors = in_images
        
    return in_images, image_tensors

def construct_classifier():
    from keras.layers import Dropout, Dense, Flatten, Input
    from keras.models import Model
    from keras import backend as K
    import tensorflow as tf
    K.set_session(tf.get_default_session())
    
    FC_SIZE = 1024
    NUM_CLASSES = 2

    in_layer = Input(shape=(1, 1, 2048,),name='input_1')
    x = Dense(FC_SIZE, activation='relu', input_dim=(1, 1, 2048,),name='dense_1')(in_layer)
    x = Flatten(name='flatten_1')(x)
    preds = Dense(NUM_CLASSES, activation='softmax', input_dim=FC_SIZE, name='classifier_output')(x)
    
    model = Model(inputs = in_layer, outputs = preds)
    
    return model

def construct_model(quantized, saved_model_dir=None, starting_weights_directory=None, is_frozen=False, is_training=True, size=64):
    from azureml.contrib.brainwave.models import Resnet50, QuantizedResnet50
    import tensorflow as tf
    from keras import backend as K
    
    # Convert images to 3D tensors [width,height,channel]
    in_images, image_tensors = preprocess_images(size=size)

    # Construct featurizer using quantized or unquantized ResNet50 model
    
    if not quantized:
        featurizer = Resnet50(saved_model_dir, is_frozen=is_frozen, custom_weights_directory = starting_weights_directory)
    else:
        featurizer = QuantizedResnet50(saved_model_dir, is_frozen=is_frozen, custom_weights_directory = starting_weights_directory)
    
    features = featurizer.import_graph_def(input_tensor=image_tensors, is_training=is_training)
    
    # Construct classifier
    with tf.name_scope('classifier'):
        classifier = construct_classifier()
        preds = classifier(features)
    
    # Initialize weights
    sess = tf.get_default_session()
    tf.global_variables_initializer().run()
    
    if not is_frozen:
        featurizer.restore_weights(sess)
    
    if starting_weights_directory is not None:
        print("loading classifier weights from", starting_weights_directory+'/class_weights_best.h5')
        classifier.load_weights(starting_weights_directory+'/class_weights_best.h5')
        
    return in_images, image_tensors, features, preds, featurizer, classifier 



In [ ]:
import glob
# for 224x224:
datadir = "./"
data_size = 224 #image width/height
n_test_file = 1
test_files = glob.glob(os.path.join(datadir, 'test_file_*'))
n_test_events = count_events(test_files)
print("n_test_events =", n_test_events)

In [ ]:
tf.reset_default_graph()
sess = tf.Session(graph=tf.get_default_graph())

with sess.as_default():
    print("Loading quantized model")
    in_images, image_tensors, features, preds, quantized_featurizer, classifier = construct_model(quantized=True, 
                                                                                                  saved_model_dir=saved_model_dir, 
                                                                                                  starting_weights_directory=custom_weights_dir_q, 
                                                                                                  is_training=False, 
                                                                                                  size=data_size)
    print(preds.name)
    pred_node_names = [preds.name[:-2]]

    K.set_learning_phase(0)
    saver = tf.train.Saver()
    tfoutpath=custom_weights_dir_q+'/tf.checkpoint'

    from tensorflow.python.framework import graph_util
    from tensorflow.python.framework import graph_io
    
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)

    f = 'constantgraph.pb.ascii'
    tf.train.write_graph(constant_graph, custom_weights_dir_q, f, as_text=True)
    print('saved the graph definition in ascii format at: ', os.path.join(custom_weights_dir_q, f))

    f = 'constantgraph.pb'
    tf.train.write_graph(constant_graph, custom_weights_dir_q, f, as_text=False)
    print('saved the graph definition in pb format at: ', os.path.join(custom_weights_dir_q, f))

    saver.save(sess, tfoutpath)